In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd 
import numpy as np

In [2]:
#Function to extract the product title
def get_title(soup):
    
    try:
        #Outer Tag object
        title = soup.find("span", attrs={"id":"productTitle"})
        
        #Inner NavigatableString Object 
        title_value = title.text
        
        #Title as a string value
        title_string = title_value.strip()
        
    except AttributeError:
        title_string = ""
        
    return title_string
        
#Function to extract Product Price
def get_price(soup):
    
    try:
        price = soup.find("span", attrs={"class":'a-price a-text-price a-size-medium'}).find("span", attrs={"class": "a-offscreen"}).text
        
    except AttributeError:
         price = ""
    return price
    
#Function to extract the product rating 
def get_rating(soup):
    
    try:
        rating = soup.find("span", attrs={"class":'a-icon-alt'}).text
    
    except AttributeError:
            rating = ""
    return rating
    
#Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()
    except AttributeError:
        review_count=""
    return review_count

#Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find('span').string.strip()
        
    except AttributeError:
        available = "Not Available"
        
    return available

In [3]:
if __name__ == '__main__':
    
    #Headers for request
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})
    
    #The webpage URL
    URL = 'https://www.amazon.com/s?k=iphone&ref=nb_sb_noss_1'
    
    #HTTP Request
    webpage = requests.get(URL, headers=HEADERS)
    
    #Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")
    
    #Fecth links as List of Tag Objects 
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})
    
    #Store the links
    links_list = []
    
    #Loop for extracting links from Tag Objects 
    for link in links:
        links_list.append(link.get('href'))
            
    d = {"title":[], "price":[], "rating":[], "reviews":[], "availability":[]}
    
    #Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)
        
        new_soup = BeautifulSoup(new_webpage.content, "html.parser")
        
        #Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))
        
amazon_df = pd.DataFrame.from_dict(d)
amazon_df['title'].replace('', np.nan, inplace=True)
amazon_df = amazon_df.dropna(subset=['title'])
amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [4]:
amazon_df

,title,price,rating,reviews,availability
0,"Apple iPhone XR, 64GB, White - Unlocked (Renew...",$267.00,4.5 out of 5 stars,"58,592 ratings",In Stock.
1,"iPhone 13 Pro Max, 128GB, Silver - Unlocked (R...",$999.00,4.0 out of 5 stars,606 ratings,In Stock.
2,"Apple iPhone 11 Pro, 64GB, Midnight Green - Un...",,4.2 out of 5 stars,"17,945 ratings",Available to ship in 1-2 days.
3,"Apple iPhone SE, 64GB, Black - Fully Unlocked ...",$221.45,4.3 out of 5 stars,"10,727 ratings",Only 4 left in stock (more on the way).
4,"iPhone 13, 128GB, Product Red - Unlocked (Rene...",$655.00,4.4 out of 5 stars,294 ratings,In Stock.
5,"Apple iPhone 12 Pro, 128GB, Graphite - Unlocke...",$646.00,4.3 out of 5 stars,142 ratings,Only 1 left in stock (more on the way).
6,"Apple iPhone 14 Pro, 256GB, Space Black - Unlo...","$1,125.00",3.5 out of 5 stars,2 ratings,Only 4 left in stock (more on the way).
7,"Apple iPhone 11 Pro Max, 64GB, Midnight Green ...",$475.00,4.3 out of 5 stars,"13,715 ratings",Only 7 left in stock (more on the way).
8,"Apple iPhone 14 Pro Max, 256GB, Space Black - ...","$1,250.00",3.9 out of 5 stars,10 ratings,Only 3 left in stock (more on the way).
9,"Apple iPhone 12 Mini, 64GB, Black - Unlocked (...",$400.00,4.3 out of 5 stars,280 ratings,Available to ship in 1-2 days.
